In [1]:
import numpy as np
import pandas as pd
from itertools import product

In [19]:
# what varies for all data sets
# n_classes_dep_var = 
# n_ind_vars =
# n_classes_ind_vars = 
# n_samples = 

def generate_data(n_classes_dep_var, n_ind_vars, n_categorical_vars, n_classes_ind_vars, n_samples):
    
    # set coefficients for the model
    coef = [3, 2, 2, 1, 1, 0.5, 0.5]
    coef = coef[:n_ind_vars]

    # generate n x p array for independent variables (predictors)
    X = np.random.uniform(0, 1, (n_samples, n_ind_vars))

    # convert independent continuous variables into categorical ones
    if n_categorical_vars == 0:
        if n_classes_ind_vars != 2:  # if no categorical vars, then do not vary number of classes
            return None
    elif n_categorical_vars >= n_ind_vars:
        return None
    else:
        for i in range(1, n_categorical_vars + 1):
            if n_classes_ind_vars == 2:
                X[:, -i] = np.where(X[:, -i] < np.quantile(X[:, -i], 0.5), 0, 1)  # 0 and 1 for binary
            elif n_classes_ind_vars == 3:
                X[:, -i] = np.where(X[:, -i] < np.quantile(X[:, -i], 0.25), 0,
                                    np.where(X[:, -i] > np.quantile(X[:, -i], 0.75), 2, 1))  # 0, 1, 2 for ternary
            
        

    # generate error term
    e = np.random.normal(0, 1, n_samples)

    # calculate y continuous
    y_continuous = 1 + np.dot(X, coef) + e

    # convert y to categorical
    # keep working from here
    if n_classes_dep_var == 2:
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.5), 'A', 'B')
    elif n_classes_dep_var == 3: 
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.25), 'A',
                                 np.where(y_continuous > np.quantile(y_continuous, 0.75), 'B', 'C'))
    elif n_classes_dep_var == 4: 
        y_categorical = np.where(y_continuous < np.quantile(y_continuous, 0.25), 'A',
                                 np.where(y_continuous < np.quantile(y_continuous, 0.5), 'B',
                                 np.where(y_continuous < np.quantile(y_continuous, 0.75), 'C', 'D')))

    # create data that is to be returned
    data_mat = np.concatenate((X, y_continuous.reshape(-1, 1), y_categorical.reshape(-1, 1)), axis = 1)
    data = pd.DataFrame(data_mat)
    # Rename columns to X1, X2, ..., y_cont, y_cat
    data.columns = [f'X{i+1}' for i in range(n_ind_vars)] + ['y_cont', 'y_cat']

    return(data)


In [20]:
# try an example
data = generate_data(3, 5, 3, 3, 100)
print(data)

                     X1                   X2   X3   X4   X5  \
0    0.6309214722829094  0.49894097240459256  1.0  0.0  2.0   
1   0.31843947712348386   0.9046706877738242  2.0  1.0  1.0   
2    0.1773337304573872    0.904201397671809  0.0  1.0  0.0   
3    0.9879366514075734   0.8194854856034515  1.0  0.0  0.0   
4    0.5372184567188039  0.11232149288947157  0.0  0.0  1.0   
..                  ...                  ...  ...  ...  ...   
95  0.15356094474441684   0.0945403659210885  1.0  2.0  2.0   
96   0.9539849436557588    0.895959068246184  0.0  1.0  0.0   
97  0.46146519001800745   0.7080906663523728  1.0  1.0  1.0   
98   0.6363784003459685  0.49879742647749004  1.0  1.0  1.0   
99   0.6677828305585315   0.7025499570091578  1.0  2.0  1.0   

                y_cont y_cat  
0    7.914467979519795     C  
1   11.359659027741415     B  
2   3.2966123001759406     A  
3    8.259571889324544     C  
4   3.8307312619999205     A  
..                 ...   ...  
95   8.210556446675575    

In [21]:
n_classes_dep_var = [2, 3, 4]
n_ind_vars = [3, 5, 7]
n_categorical_vars = np.arange(7)
n_classes_ind_vars = [2, 3]
n_samples = [100, 500, 1000, 10000]

In [18]:
# Generate and save datasets
np.random.seed(3103)
for n_classes_dep, n_ind, n_categorical, n_classes_ind, n_samp in product(n_classes_dep_var, n_ind_vars, n_categorical_vars, n_classes_ind_vars, n_samples):
    data = generate_data(n_classes_dep, n_ind, n_categorical, n_classes_ind, n_samp)
    if data is not None:  # Avoid saving if no data is generated
        filename = f'../dat/{n_categorical}_{n_classes_ind}_{n_ind}_{n_classes_dep}_{n_samp}_samples.csv'
        data.to_csv(filename, index=False)